In [87]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '.'))

In [128]:
import pandas as pd
from datetime import date
from src import data

In [111]:
# Transactions 
# =================================================================
orders = [
  {
    'asset': 'COUR',
    'order_date': date(2021, 4, 5),
    'order_id': 1,
    'order_units': 16,
    'unit_cost_usd': 10,
  },
  {
    'asset': 'COUR',
    'order_date': date(2021, 4, 5),
    'order_id': 2,
    'order_units': 1,
    'unit_cost_usd': 5,
  },
]

# Daily Day-End Prices
# =================================================================
prices = [
  {
    'price_date': date(2021, 4, 5),
    'asset': 'COUR',
    'unit_price_usd': 5
  },
  {
    'price_date': date(2021, 4, 6),
    'asset': 'COUR',
    'unit_price_usd': 10
  }
]

In [112]:
orders_df = (
  pd
    .DataFrame(orders)
    .assign(
      order_cost_usd = lambda df: df['order_units'] * df['unit_cost_usd']
    )
)
orders_df

,asset,order_date,order_id,order_units,unit_cost_usd,order_cost_usd
0,COUR,2021-04-05,1,16,10,160
1,COUR,2021-04-05,2,1,5,5


In [113]:
prices_df = pd.DataFrame(prices)
prices_df

,price_date,asset,unit_price_usd
0,2021-04-05,COUR,5
1,2021-04-06,COUR,10


In [114]:
market_data_df = (
  get_market_data(['COUR', 'BB.TO'], interval = '1d')
)

[*********************100%***********************]  2 of 2 completed


/Users/calvindelima/projects/investing/.venv/lib/python3.9/site-packages/pandas/core/generic.py:4152: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [130]:
prices_df = (
  ((market_data_df['adjusted_close'] + market_data_df['open']) / 2)
    # Move date to columns
    .reset_index()
    .melt(
      id_vars = ['date'],
      value_name = 'unit_price_usd',
      var_name = 'asset'
    )
    .rename(columns = {'date': 'price_date'})
)

prices_df.head()

,price_date,asset,unit_price_usd
0,1997-12-15,BB.TO,1.112500
1,1997-12-16,BB.TO,1.112500
2,1997-12-17,BB.TO,1.095833
3,1997-12-18,BB.TO,1.066667
4,1997-12-19,BB.TO,1.033333


In [129]:
order_value_snapshot_df = (
  prices_df
    # Cross-join on the orders to get one row per order
    # per day
    .merge(
      orders_df[['asset', 'order_date', 'order_id', 'order_units', 'order_cost_usd']],
      how = 'inner',
      on = ['asset'],
      indicator = True
    )
    # Only include records where the `price_date` is on or after the
    # `order_date`
    .query('order_date <= price_date')
    .assign(
      order_value_usd = lambda df: df['order_units'] * df['unit_price_usd']
    )
    [[
      'asset',
      'price_date',
      'order_date',
      'order_id',
      'order_units',
      'unit_price_usd',
      'order_cost_usd',
      'order_value_usd'
    ]]
    # Since we melted the market data dataframe, we'll have one
    # record for every date from the min date to the max date
    # over all assets, so lots of NULL unit_price_usd values in
    # some cases
    .dropna(subset = ['unit_price_usd'])
)
order_value_snapshot_df

,asset,price_date,order_date,order_id,order_units,unit_price_usd,order_cost_usd,order_value_usd
11708,COUR,2021-04-05,2021-04-05,1,16,49.335001,160,789.360016
11709,COUR,2021-04-05,2021-04-05,2,1,49.335001,5,49.335001
11710,COUR,2021-04-06,2021-04-05,1,16,53.869999,160,861.919983
11711,COUR,2021-04-06,2021-04-05,2,1,53.869999,5,53.869999
11712,COUR,2021-04-07,2021-04-05,1,16,57.650000,160,922.399994
11713,COUR,2021-04-07,2021-04-05,2,1,57.650000,5,57.650000
11714,COUR,2021-04-08,2021-04-05,1,16,57.000000,160,912.000000
11715,COUR,2021-04-08,2021-04-05,2,1,57.000000,5,57.000000
11716,COUR,2021-04-09,2021-04-05,1,16,53.830000,160,861.279999
11717,COUR,2021-04-09,2021-04-05,2,1,53.830000,5,53.830000


In [125]:
asset_value_snapshot_df = (
  order_value_snapshot_df
    .groupby(['asset', 'price_date'])
    .agg(
      asset_cost_usd = pd.NamedAgg('order_cost_usd', 'sum'),
      asset_value_usd = pd.NamedAgg('order_value_usd', 'sum')
    )
    .reset_index()
    [[
      'asset',
      'price_date',
      'asset_cost_usd',
      'asset_value_usd'
    ]]
    .rename(columns = {'price_date': 'snapshot_date'})
)
asset_value_snapshot_df

,asset,snapshot_date,asset_cost_usd,asset_value_usd
0,COUR,2021-04-05,165,838.695017
1,COUR,2021-04-06,165,915.789982
2,COUR,2021-04-07,165,980.049994
3,COUR,2021-04-08,165,969.000000
4,COUR,2021-04-09,165,915.109999
5,COUR,2021-04-12,165,886.975019
6,COUR,2021-04-13,165,863.854977
7,COUR,2021-04-14,165,840.310005
8,COUR,2021-04-15,165,828.325006
9,COUR,2021-04-16,165,800.700006


In [127]:
portfolio_value_snapshot_df = (
  asset_value_snapshot_df
    .groupby(['snapshot_date'])
    .agg(
      portfolio_cost_usd = pd.NamedAgg('asset_cost_usd', 'sum'),
      portfolio_value_usd = pd.NamedAgg('asset_value_usd', 'sum'),
    )
    .reset_index()
)
portfolio_value_snapshot_df

,snapshot_date,portfolio_cost_usd,portfolio_value_usd
0,2021-04-05,165,838.695017
1,2021-04-06,165,915.789982
2,2021-04-07,165,980.049994
3,2021-04-08,165,969.000000
4,2021-04-09,165,915.109999
5,2021-04-12,165,886.975019
6,2021-04-13,165,863.854977
7,2021-04-14,165,840.310005
8,2021-04-15,165,828.325006
9,2021-04-16,165,800.700006


In [ ]:
orders